Clase 9
===

* Resolver el problema primal-dual

Resumen del problema original (minimizar norma Euclideana)
------------------------------------------------------------------

Usando la teoria de dualidad hemos reducido la optimización en dos niveles:

\begin{align}
\mbox{min}\ & \sum_{i \in I} ||x_i-y_i|| \\
\mbox{s.a.} & \\
&\mbox{max}\  f(x)=c^Tx \\
&\ \mbox{s.a.}  \\
&\ Ax=0 \\
&\ LB<=x<=UB
\end{align}

en una optimización simple:

\begin{align}
\mbox{min}\ & ||M\bar{x}-\bar{x}^o||^2 \\
\mbox{s.a.} & \\
& \bar{c}^T\bar{x} -b^Ty =0 \\
&\ \bar{A}\bar{x}=b \\
&\ \bar{A}^Ty \le \bar{c} \\
&\ \bar{x} \ge 0
\end{align}

Como ejemplo, la (mini) red metabólica mostrada en la figura de abajo:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/miniRed_clase8.png" alt="Drawing" style="width: 200px;"/>

Resulta en:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/clase8_miniRed2019.jpg" alt="Drawing" style="width: 600px;"/>

Con lo cual ya tenemos todos los elementos para construir el problema primal-dual. Pero, ¿cómo se construyen todas esas nuevas matrices y vectores? Que el computador haga el trabajo por nosotros.

In [1]:
import numpy as np

def hacerPrimal(A,c,x_l,x_u):
    # get number of (M)etabolites and (R)eactions
    M,R = A.shape
    # define empty matrices with the appropiate dimensions
    c_bar = np.zeros((2*R, 1 ))
    A_bar = np.zeros((M+R,2*R))
    b = np.zeros((M+R, 1 ))
    # fill matrices c_bar,A_bar,b
    c_bar[:R] = -c
    A_bar[:M,:R] = A
    A_bar[M:,:R] = np.identity(R)
    A_bar[M:,R: ] = np.identity(R)
    b[:M] = -A*x_l
    b[M:] = x_u-x_l
    return c_bar,A_bar,b

c=np.matrix([0,0,0,1]).reshape((4,1))
A=np.matrix([[1,-1,-1,0],[0,1,1,-1]])
x_l=np.matrix([-1000]*4).reshape((4,1))
x_u=np.matrix([10]+[1000]*3).reshape((4,1))
c_bar,A_bar,b = hacerPrimal(A,c,x_l,x_u)

Resolución en python
-----------------------

El problema resulta en una optimización cuadrática la su forma canónica es:

\begin{align}
\mbox{min}\ & 1/2\bar{\bar{x}}^TP\bar{\bar{x}}-q\bar{\bar{x}} \\
\mbox{s.a.} & \\
&\ \bar{\bar{A}}\bar{\bar{x}}=\bar{b} \\
& G\bar{\bar{x}} \le h
\end{align}

En donde la función objetivo resulta de expander el cuadrado de la norma Euclideana:

\begin{align}
\mbox{min}\ ||M\bar{\bar{x}}-\bar{\bar{x}}^o||^2 &= \mbox{min}\ 1/2\bar{\bar{x}}^TP\bar{\bar{x}}+q^T\bar{\bar{x}} \\
\mbox{En donde:} \\
&P=(M^TM) \\
&q=(-M^T\bar{\bar{x}}^o)
\end{align}


Para resolver este tipo de problemas podemos instalar [CVXOPT](http://cvxopt.org/userguide/intro.html), un paquete para minimizar funciones objetivos cuadráticas (como es el caso de la norma Euclideana) sujeto a restricciones de igualdad y desigualdad. Esto se puede hacer abriendo el terminal de anaconda e ingresando el siguiente comando: `pip install cvxopt`.

Ejercicio
-----------

Resolver el siguiente problema (original [aquí](https://scaron.info/blog/quadratic-programming-in-python.html)) con cvxopt:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/quadratic_example_clase9.png" alt="Drawing" style="width: 400px;"/>

In [2]:
# Load libraries
import cvxopt
from cvxopt import matrix
import numpy as np
# Defines a wraper for cvxopt 
# Among other things it transforms matrices from numpy to cvxopt format
def cvxopt_solve_qp(P, q, G=None, h=None, A=None, b=None):
    P = .5 * (P + P.T)  # make sure P is symmetric
    args = [matrix(P), matrix(q)]
    if G is not None:
        args.extend([matrix(G), matrix(h)])
        if A is not None:
            args.extend([matrix(A), matrix(b)])
    sol = cvxopt.solvers.qp(*args)
    if 'optimal' not in sol['status']:
        return None
    return np.array(sol['x']).reshape((P.shape[1],))
# Define matrices    
M = np.array([[1., 2., 0.], [-8., 3., 2.], [0., 1., 1.]])
P = np.dot(M.T, M)
q = np.dot(np.array([3., 2., 3.]), -M).reshape((3,))
G = np.array([[1., 2., 1.], [2., 0., 1.], [-1., 2., -1.]])
h = np.array([3., 2., -2.]).reshape((3,))
# The results:
cvxopt_solve_qp(P, q, G, h)

     pcost       dcost       gap    pres   dres
 0: -1.0062e+01 -8.2131e+00  3e+00  8e-01  6e-17
 1: -8.9877e+00 -7.1656e+00  6e-01  3e-01  2e-16
 2: -4.7428e+00 -5.6786e+00  9e-01  1e-16  1e-15
 3: -5.5832e+00 -5.5940e+00  1e-02  5e-17  4e-16
 4: -5.5921e+00 -5.5922e+00  1e-04  2e-16  3e-16
 5: -5.5922e+00 -5.5922e+00  1e-06  1e-16  3e-16
Optimal solution found.


array([ 0.12997344, -0.06498685,  1.74005307])

Tarea
---------

Resolver el problema de la red métabolica.

1. Escribir una función que tome como argumentos $c$, $A$, $x^u$, y $x^l$, entregue como resultado $G$, $h$, $\bar{\bar{A}}$, $\bar{b}$.
2. Asumiendo que los datos observados son $x_1=10$, y $x_2=3$, definir $M$,$x^o$, y $P$, y $q$.
3. Utilizar cvxopt para encontrar la solución.

In [8]:
# import numpy as np

# def hacerPrimalDual(A,c,x_l,x_u,x_o):
#     # get number of (M)etabolites and (R)eactions
#     M,R = A.shape
#     # define empty matrices with the appropiate dimensions
#     c_bar = np.zeros((2*R, 1 ))
#     A_bar = np.zeros((M+R,2*R))
#     b = np.zeros((M+R, 1 ))
#     # fill matrices c_bar,A_bar,b
#     c_bar[:R] = -c
#     A_bar[:M,:R] = A
#     A_bar[M:,:R] = np.identity(R)
#     A_bar[M:,R: ] = np.identity(R)
#     b[:M] = -A*x_l
#     b[M:] = x_u-x_l
#     # fill matrices A_bbar,b_bar, G, and h
#     A_bbar = np.zeros((M+R+1,M+3*R))
#     b_bar  = np.zeros((M+R+1,1))
#     G      = np.zeros((2*R,M+3*R))
#     h      = c_bar
#     # fill matrices A_bbar,b_bar,G
#     A_bbar[0 ,:2*R ] = np.transpose(c_bar) 
#     A_bbar[0 , 2*R:] = -np.transpose(b)
#     A_bbar[1:,:2*R ] = A_bar
#     b_bar[1:,:]      = b
#     G[:,2*R:]        = np.transpose(A_bar) 
#     # create objetive function matrices M(MM) and q
#     MM = np.zeros((M+3*R,M+3*R))
#     MM[0,0] = 1
#     MM[2,2] = 1
#     x_o_bar=np.zeros((M+3*R,1))
#     x_o_bar[0,0] = x_o[0,0] - x_l[0,0]
#     x_o_bar[2,0] = x_o[2,0] - x_l[2,0]
#     q=np.dot(-MM,x_o_bar).reshape((M+3*R,))
#     # Define matrices P and q
#     P = np.dot(MM.T, MM)
#     return c_bar,A_bar,b,A_bbar,b_bar,G,h, MM, P, q

# c=np.matrix([0,0,0,1]).reshape((4,1))
# A=np.matrix([[1,-1,-1,0],[0,1,1,-1]])
# x_l=np.matrix([-1000]*4).reshape((4,1))
# x_u=np.matrix([10]+[1000]*3).reshape((4,1))
# x_o=np.zeros((4,1))
# x_o[0,0]=10 #x1
# x_o[2,0]=7  #x3
# c_bar,A_bar,b,A_bbar,b_bar,G,h,M,P,q = hacerPrimalDual(A,c,x_l,x_u,x_o)

# # The results:
# cvxopt_solve_qp(P, q, G, h,A_bbar,b_bar)

In [7]:
import numpy as np

def hacerPrimalDual(A,c,x_l,x_u):
    # get number of (M)etabolites and (R)eactions
    M,R = A.shape
    # define empty matrices with the appropiate dimensions
    c_bar = np.zeros((2*R, 1 ))
    A_bar = np.zeros((M+R,2*R))
    b = np.zeros((M+R, 1 ))
    # fill matrices c_bar,A_bar,b
    c_bar[:R] = -c
    A_bar[:M,:R] = A
    A_bar[M:,:R] = np.identity(R)
    A_bar[M:,R: ] = np.identity(R)
    b[:M] = -A*x_l
    b[M:] = x_u-x_l
    # fill matrices A_bbar,b_bar, G, and h
    A_bbar = np.zeros((M+R+1,M+3*R))
    b_bar  = np.zeros((M+R+1,1))
    G      = np.zeros((4*R,M+3*R))
    h      = np.zeros((4*R,1))
        # fill matrices A_bbar,b_bar,G
    A_bbar[0 ,:2*R ] = np.transpose(c_bar) 
    A_bbar[0 , 2*R:] = -np.transpose(b)
    A_bbar[1:,:2*R ] = A_bar
    b_bar[1:,:]      = b
    G[:2*R,2*R:]     = np.transpose(A_bar) 
    G[2*R:,:2*R]     = -np.identity(2*R) 
    h[:2*R,:]        = c_bar
    return c_bar,A_bar,b,A_bbar,b_bar,G,h

###################################################################
# Given c, A, x_l, x_u 
c=np.matrix([0,0,0,1]).reshape((4,1))
A=np.matrix([[1,-1,-1,0],[0,1,1,-1]])
x_l=np.matrix([-1000]*4).reshape((4,1))
x_u=np.matrix([10]+[1000]*3).reshape((4,1))
# get c_bar,A_bar,b,A_bbar,b_bar,G,h
c_bar,A_bar,b,A_bbar,b_bar,G,h = hacerPrimalDual(A,c,x_l,x_u)
###################################################################
# Given observed fluxes, x_o, get P and q
x_o=np.zeros((4,1))
x_o[0,0]=10 #x1
x_o[2,0]=3  #x3
# create objetive function matrices M, q, and x_o_bar
M,R = A.shape
MM = np.zeros((M+3*R,M+3*R))
MM[0,0] = 1
MM[2,2] = 1
x_o_bar=np.zeros((M+3*R,1))
x_o_bar[0,0] = x_o[0,0] - x_l[0,0]
x_o_bar[2,0] = x_o[2,0] - x_l[2,0]
# Define matrices P and q
P = np.dot(MM.T, MM)
q=np.dot(-MM,x_o_bar).reshape((M+3*R,))


# The results:
cvxopt_solve_qp(P, q, G, h,A_bbar,b_bar)

     pcost       dcost       gap    pres   dres
 0: -9.9682e+05 -2.3073e+06  1e+06  5e+00  4e+00
 1: -1.0044e+06 -1.0892e+06  9e+04  3e-01  3e-01
 2: -1.0100e+06 -1.0353e+06  3e+04  9e-02  8e-02
 3: -1.0110e+06 -1.0297e+06  2e+04  6e-02  6e-02
 4: -1.0121e+06 -1.0224e+06  1e+04  4e-02  3e-02
 5: -1.0129e+06 -1.0171e+06  5e+03  2e-02  1e-02
 6: -1.0131e+06 -1.0132e+06  1e+02  4e-04  3e-04
 7: -1.0131e+06 -1.0131e+06  1e+00  4e-06  3e-06
 8: -1.0131e+06 -1.0131e+06  1e-02  4e-08  3e-08
Optimal solution found.


array([  1.00999997e+03,   1.00699997e+03,   1.00300000e+03,
         1.00999997e+03,   2.86536989e-05,   9.93000028e+02,
         9.97000001e+02,   9.90000029e+02,   9.99999991e-01,
         9.99999994e-01,  -9.99999984e-01,   2.12371216e-09,
         2.12371219e-09,   7.14245170e-10])